In [3]:
#import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
gpd.options.use_pygeos = False
import seaborn as sns
import os
import numpy as np

In [5]:
from shapely.geometry import Point, Polygon
import shapely
shapely.speedups.disable()

In [ ]:
nb = os.path.join('Resources',"Neighbourhoods","Neighbourhoods-Copy1.shp")
regions = gpd.read_file(nb)
regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()
regions.head()

In [ ]:
regions['geometry']

In [ ]:
#build dataframe
lights_df = pd.read_csv('Resources/Red Light Cameras Data.csv')
lights_df.head()

In [ ]:
# Create a set of starbucks locations lat and lng combinations
lng_lats = []

lngs = lights_df['LONGITUDE']
lats = lights_df['LATITUDE']

lng_lats = zip(lngs, lats)

neighbourhood_id = []
neighbourhood_name = []
lats_list = []
lngs_list = []

for lng_lat in lng_lats:
    point = Point(lng_lat[0], lng_lat[1])
    
    for i in np.arange(len(regions)):
        poly = regions.loc[i, 'geometry']
        
        if point.within(poly):
            neighbourhood_id.append(regions.loc[i, 'FIELD_6'])
            neighbourhood_name.append(regions.loc[i, 'FIELD_7'])
            lats_list.append(lng_lat[1])
            lngs_list.append(lng_lat[0])

In [ ]:
lights_dict = {
    'Lat' : lats_list,
    'Long' : lngs_list,
    'Hood ID' : neighbourhood_id,
    'Neighbourhood' : neighbourhood_name
}

clean_lights = pd.DataFrame(lights_dict)

clean_lights

In [ ]:
lights_val_counts = clean_lights['Neighbourhood'].value_counts()

In [ ]:
# number of red lights each location
lights_val_counts = lights_val_counts.reset_index()
lights_val_counts.columns = ['Neighbourhood', 'Number of Red Lights']

lights_val_counts

In [ ]:
clean_lights_df = clean_lights.drop_duplicates(subset='Neighbourhood', keep='first')
clean_lights_df

In [ ]:
lights_final = pd.merge(lights_val_counts, clean_lights_df, on='Neighbourhood', how='inner')
lights_final.head()

In [ ]:
lights_final.drop(['Lat', 'Long'], axis='columns', inplace=True)
lights_final = lights_final[['Hood ID','Neighbourhood','Number of Red Lights']]
lights_final.rename(columns={"Hood ID": "Hood_ID"}, inplace=True)

In [ ]:
lights_final